In [1]:
import pandas as pd
import numpy as np
from copy import deepcopy
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.model_selection import KFold, GridSearchCV, train_test_split, cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.metrics import make_scorer, mean_squared_error, mean_absolute_error, r2_score
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from torch.optim import AdamW
from torch.optim.lr_scheduler import ReduceLROnPlateau, OneCycleLR
from sklearn.preprocessing import StandardScaler
from collections import Counter
import random
import matplotlib.pyplot as plt
from tqdm import tqdm


In [2]:
datas_folder = "clean_datas/"
train_data_file = "ecfp_train.csv"
test_data_file = "ecfp_test.csv"
cddd_train_file = "cddd_train.csv"
cddd_test_file =  "cddd_test.csv"
mix_train_file = "mix_train.csv"
mix_test_file =  "mix_test.csv"
train = pd.read_csv(datas_folder + train_data_file)
test = pd.read_csv(datas_folder + test_data_file)
cddd_train = pd.read_csv(datas_folder + cddd_train_file)
cddd_test = pd.read_csv(datas_folder + cddd_test_file)
mix_train = pd.read_csv(datas_folder + mix_train_file)
mix_test = pd.read_csv(datas_folder + mix_test_file)



In [3]:
def custom_loss(y_true, y_pred):
    y_pred_exp = np.exp(y_pred)
    loss = mean_squared_error(np.exp(y_true), y_pred_exp) 
    return loss
custom_scorer = make_scorer(custom_loss, greater_is_better=False)

def prep_fitt(datas):
    datas = datas.copy()
    try:
        return datas.drop(labels=["RT", "SMILES", "mol", "Compound"], axis=1)
    except Exception:
        return datas.drop(labels=["SMILES", "mol", "Compound"], axis=1)
    
    
# CDDD_datas
#y = cddd_train["RT"]
#X = prep_fitt(cddd_train)
#X_test = prep_fitt(cddd_test)


# ECFP_datas
#y = train["RT"]
#X = prep_fitt(train)
#X_test = prep_fitt(test)


# MIXED_datas
#scaler = StandardScaler()
y = mix_train["RT"]
X = prep_fitt(mix_train)
X_test = prep_fitt(mix_test)
# col_std = [colonne for colonne in X.columns if 'cddd' in colonne.lower()]
# X_to_std = X[col_std]
# scaler = StandardScaler()
# X_to_std = scaler.fit_transform(X_to_std)
# X[col_std] = X_to_std
# col_std = [colonne for colonne in X_test.columns if 'cddd' in colonne.lower()]
# X_to_std = X_test[col_std]
# X_to_std = scaler.transform(X_to_std)
# X_test[col_std] = X_to_std
#X_test = pd.DataFrame(scaler.transform(X_test))

In [4]:

seed_num = 31
torch.use_deterministic_algorithms(True)
random.seed(seed_num)
np.random.seed(seed_num)
torch.manual_seed(seed_num)
def seed_worker(worker_id):
    worker_seed = torch.initial_seed() % 2**32
    np.random.seed(worker_seed)
    random.seed(worker_seed)
g = torch.Generator()
g.manual_seed(seed_num)
print(" ")

# FNN Model 2 Hidden-Layers

In [5]:
class DNNModel(nn.Module):
    def __init__(self, input_size, n1, n2, activation, batch_normal = False ,dropout_rate = None):
        super(DNNModel, self).__init__()
        self.batch_normal = batch_normal
        self.fc1 = nn.Linear(input_size, n1)
        
        if batch_normal : 
            self.bn1 = nn.BatchNorm1d(n1)
            #print("--- std batch -----")
            
        self.activation1 = nn.ReLU() if activation == 'relu' else nn.Tanh() if activation == 'tanh' else nn.Sigmoid()
        self.fc2 = nn.Linear(n1, n2)
        
        if batch_normal : 
            self.bn2 = nn.BatchNorm1d(n2) 
        
        self.activation2 = nn.ReLU() if activation == 'relu' else nn.Tanh() if activation == 'tanh' else nn.Sigmoid() 
        self.fc3 = nn.Linear(n2, 1)
        if type(dropout_rate) == float : 
            self.dropout = nn.Dropout(dropout_rate)
            self.drop_or_no_drop = True
            #print(f"\n--- dropout: {dropout_rate} --- \n")
        else : 
            self.drop_or_no_drop = False
    

    def forward(self, x):  
        if self.batch_normal:  # BATCH NORMAL
            if self.drop_or_no_drop:  # DROPOUT
                x = self.activation1(self.bn1(self.fc1(x)))
                x = self.dropout(x)
                x = self.activation2(self.bn2(self.fc2(x)))
                x = self.dropout(x)
                x = self.fc3(x)
                return x
            else :      # NO DROPOUT
                x = self.activation1(self.bn1(self.fc1(x)))
                x = self.activation2(self.bn2(self.fc2(x)))
                x = self.fc3(x)
                return x
        
        else :  # NO BATCH NORMAL
            if self.drop_or_no_drop: # DROPOUT
                x = self.activation1(self.fc1(x))
                x = self.dropout(x)
                x = self.activation2(self.fc2(x))
                x = self.dropout(x)
                x = self.fc3(x)
                return x
            else :          # NO DROPOUT
                x = self.activation1(self.fc1(x))
                x = self.activation2(self.fc2(x))
                x = self.fc3(x)
                return x    

### Train / Test function

In [6]:
def train(model, train_loader, optimizer, criterion, scheduler = None): 
    model.train()
    train_loss = 0.0
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        #train_loss += np.sqrt(loss.item())
        train_loss += loss.item()
        
        if scheduler != None :
            scheduler.step()
    return train_loss / len(train_loader)


def test(model, test_loader, criterion) : 
    model.eval()   
    test_loss = 0.0
    rmse_test_loss = 0.0
    with torch.no_grad():
        inputs, labels = test_loader[:]
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        test_loss = loss.item()
        
        # for inputs, labels in test_loader:
            
        #     outputs = model(inputs)
        #     loss = criterion(outputs, labels)
        #     rmse_test_loss += np.sqrt(loss.item())
        #     test_loss += loss.item()
    
    return [test_loss, np.sqrt(test_loss)]


### Simple Validation with monitoring

In [ ]:
def simple_validation(X, y, params, learning_rate = 0.001, test_size = 0.5, lr_modifier = False, batch_normal = False, verbose = False, nested_cross_validation = False, y_val = None, X_val = None) : 
    n1, n2, activation = params["n1"], params["n2"], params["activation"]
    try : 
        num_epochs =  params["num_epochs"]
    except : 
        num_epochs=450
    try : 
        batch_size = params["batch_size"]
    except: 
        batch_size= None
    try : 
        dropout_rate = params["dropout_rate"]
    except : 
        dropout_rate = None
    try : 
        lr_max = params["lr_max"]
    except : 
        lr_max = 0.003
    try : 
        opti = params["opti"]
        #print(activation)
    except : 
        opti = "adam"
        
    
    if not nested_cross_validation : 
        X = torch.tensor(X.to_numpy(), dtype=torch.float32)
        y = torch.tensor(y.to_numpy(), dtype=torch.float32).view(-1, 1)
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=42, shuffle = True)  
    else : 
        y_test = y_val
        X_test = X_val
        X_train = X
        y_train = y
        
    train_loader = TensorDataset(X_train, y_train)
    test_loader = TensorDataset(X_test,y_test)
    
    if batch_size == None : 
        #print("Attention no batch \n")
        train_loader = DataLoader(train_loader, batch_size=len(X_train), shuffle=True, worker_init_fn= seed_worker, generator=g)
    else : 
        #print(f"- batch size: {batch_size}\n")
        train_loader = DataLoader(train_loader, batch_size=batch_size, shuffle=True, worker_init_fn= seed_worker, generator=g)
    
        
    model = DNNModel(input_size= X_train.shape[1],  n1=n1,  n2=n2, activation=activation, dropout_rate=dropout_rate, batch_normal=batch_normal)
    criterion = nn.MSELoss()
    
    if opti == "adam" : 
        optimizer = optim.Adam(model.parameters(), lr= learning_rate) 
    
    else : 
        optimizer = AdamW(model.parameters(), lr=0.001, weight_decay= 0.01)
    
    
    if lr_modifier :
        scheduler = OneCycleLR(optimizer, max_lr= lr_max, total_steps=num_epochs * len(train_loader))

    prev_lr = None
    train_losses = []
    test_losses = []
    rmse_loss = []
    epochs_list = list(range(1, num_epochs + 1))
    lr_list = []
    
    

    for epoch in tqdm(range(num_epochs),disable = verbose,desc = "Trainig Epochs",unit ="epoch" ,colour = "green") : 
    
        if lr_modifier :                
            train_loss = train(model, train_loader, criterion= criterion, optimizer=optimizer, scheduler= scheduler)
        else : 
            train_loss = train(model, train_loader, criterion= criterion, optimizer=optimizer)
        
        train_losses.append(train_loss)
        
        test_loss = test(model, test_loader, criterion=criterion)
        test_losses.append(test_loss[0])
        rmse_loss.append(test_loss[1])
        
        if lr_modifier :
            current_lr = scheduler.optimizer.param_groups[0]['lr']
            lr_list.append(current_lr)
            
            if current_lr != prev_lr:  
                prev_lr = current_lr
            if verbose and not nested_cross_validation: 
                print(f'Epoch {epoch + 1}/{num_epochs}, Train Loss: {train_loss:.4f}, Test Loss: {test_loss[0]:.4f}, Learning Rate: {current_lr} ')
        else : 
            if verbose and not nested_cross_validation :
                print(f"Epoch {epoch + 1}/{num_epochs}, Train Loss: {train_loss:.4f}, Test Loss: {test_loss[0]:.4f}")
        
    model.eval()   
    with torch.no_grad():
            prediction = model(X_test)
            training = model(X_train)
        
    rmse_test = np.sqrt(mean_squared_error(y_test, prediction))
    rmse_train = np.sqrt(mean_squared_error(y_train,training))
    if nested_cross_validation : 
        return [rmse_train, rmse_test] 
    
    else:
        print(f"Final  test error : {rmse_test} | Final train error : {rmse_train} | Min test error : {np.min(rmse_loss):.4f}")  
        fig, ax = plt.subplots(figsize=(10, 6))
        i = test_losses.index(min(test_losses))
        print(f"Epoch for minimum test loss : {epochs_list[i]}")
        min_test_loss_value = test_losses[i]
        min_test_loss_epoch = epochs_list[i]
        ax.axhline(min_test_loss_value, linestyle='--', color='white', label=f'Min Test Loss: {min_test_loss_value:.4f} | epoch: {min_test_loss_epoch}')
        ax.plot(epochs_list, train_losses, label='Train RMSE', c='#00C3FF')
        ax.plot(epochs_list,test_losses, color="#FF9200", label='Test RMSE')
        
        if lr_modifier : 
            ax2 = ax.twinx()
            ax2.set_ylabel('Learning Rate', color="white")
            ax2.plot(epochs_list, lr_list, color="#90FF15", label="Learning Rate")
            ax2.tick_params(axis='y', colors="white")
            ax2.legend(loc='upper center', facecolor='#2F2F2F', edgecolor='white', labelcolor='white')
            ax2.spines['bottom'].set_color('white')  
            ax2.spines['top'].set_color('white')    
            ax2.spines['right'].set_color('white')   
            ax2.spines['left'].set_color('white')    
            ax2.xaxis.label.set_color('white')
            ax2.yaxis.label.set_color('white')
            
        #ax.grid(color='#E27429', linestyle='--', linewidth=0.5)
        ax.set_facecolor('#2F2F2F') 
        ax.spines['bottom'].set_color('white')  
        ax.spines['top'].set_color('white')    
        ax.spines['right'].set_color('white')   
        ax.spines['left'].set_color('white')    
        ax.xaxis.label.set_color('white')
        ax.yaxis.label.set_color('white')
        ax.tick_params(axis='x', colors='white')
        ax.tick_params(axis='y', colors='white')
        fig.patch.set_facecolor('#2F2F2F')
        ax.legend(loc='upper right', facecolor='#2F2F2F', edgecolor='white', labelcolor='white')
        ax.set_xlabel('Epoch')
        ax.set_ylabel('Loss (MSE)')
        #ax.set_ylim(0, 2.5)
        #ax.set_ylim(0, np.max(test_losses))
        ax.set_ylim(0,5)
        
        plt.figtext(0.6, 0.63,  
                f" Final train loss: {train_losses[-1]:.4f} | RMSE = {rmse_train:.4f} \n Final test loss: {test_losses[-1]:.4f} | RMSE =  {(rmse_test):.4f} \n Min test RMSE: {np.min(rmse_loss):.4f}", 
                horizontalalignment ="left", 
                wrap = True, fontsize = 10,  
                bbox ={'facecolor':'grey',  
                    'alpha':0.25, 'pad':5},
                color = "white")
                
        fig, ax = plt.subplots(figsize=(8, 6))
        ax.scatter(y_test, prediction, c="#235a8c", label='predictions')
        ax.plot(y, y, color="#FF9200", label='objective')
        #ax.grid(color='white', linestyle='--', linewidth=0.5)
        ax.errorbar(x=(min(y) + max(y)) / 2, y=(min(y) + max(y)) / 2,
                yerr=rmse_test/2, color='#c9312c', fmt='|', markersize=5, capsize=3, label='RMSE')
        
        
        ax.set_facecolor('#2F2F2F') 
        ax.spines['bottom'].set_color('white')  
        ax.spines['top'].set_color('white')    
        ax.spines['right'].set_color('white')   
        ax.spines['left'].set_color('white')    
        ax.xaxis.label.set_color('white')
        ax.yaxis.label.set_color('white')
        ax.tick_params(axis='x', colors='white')
        ax.tick_params(axis='y', colors='white')
        fig.patch.set_facecolor('#2F2F2F')
        ax.legend(loc='upper left', facecolor='#2F2F2F', edgecolor='white', labelcolor='white')
        ax.set_xlabel('True Values')
        ax.set_ylabel('Predictions')
        ax.set_title(f'RMSE = {rmse_test}', color='white')
        plt.show()
        
    
        return 0
    
param = {'n1': 1024, 'n2': 2048, 'activation': 'tanh', "num_epochs" :350, "batch_size" : 64, "dropout_rate" : 0.3, "opti" : "adamW", "lr_max" : 0.003}  
simple_validation(X,y,param, test_size= 0.1, learning_rate = 0.001, lr_modifier= True, batch_normal= False, verbose=False,)  




### Sumbmission function 

In [ ]:

def submission(X,y,X_test,params,file_name,scheduler = False, learning_rate = 0.001) : 
    
    n1, n2, activation = params["n1"], params["n2"], params["activation"]
    try : 
        num_epochs = params["num_epochs"]
    except : 
        num_epochs=450
    try : 
        batch_size = params["batch_size"]
    except: 
        batch_size= None
    try : 
        dropout_rate = params["dropout_rate"]
    except : 
        dropout_rate = None
    
    X = torch.tensor(X.to_numpy(), dtype=torch.float32)
    y = torch.tensor(y.to_numpy(), dtype=torch.float32).view(-1, 1)
    X_test = torch.tensor(X_test.to_numpy(), dtype=torch.float32)
   
    train_loader = TensorDataset(X, y)
    train_loader = DataLoader(train_loader, batch_size= batch_size, shuffle=True, worker_init_fn= seed_worker, generator=g)
     
    model = DNNModel(X.shape[1], n1, n2, activation, dropout_rate= dropout_rate)
    criterion = nn.MSELoss()
    optimizer = AdamW(model.parameters(), lr=learning_rate, weight_decay= 0.01)
    #optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    
    if scheduler :
        scheduler = OneCycleLR(optimizer, max_lr=0.003, total_steps= num_epochs * len(train_loader))
    
    else :
        scheduler = None
        
    for epoch in tqdm(range(num_epochs), desc= "Training Epochs", unit = "epoch", colour = "green") : 
        train(model=model , train_loader=train_loader , optimizer=optimizer , criterion=criterion , scheduler=scheduler)


    model.eval()
    with torch.no_grad():
        prediction = model(X_test)
        
    sub_dataframe = pd.DataFrame(list(enumerate(prediction.numpy(), start=1)), columns=['ID', 'RT']).astype(float)
    submission_name = f'prediction/{file_name}.csv'     
    sub_dataframe.to_csv(submission_name, index=False)
    torch.save(model.state_dict(), f'model/{file_name}.pth')
    print("_________________\n| Prediction Saved :D ")
    

param = {'n1': 1024, 'n2': 2048, 'activation': 'tanh', "num_epochs" :350, "batch_size" : 64, "dropout_rate" : 0.3, "opti" : "adamW", "lr_max" : 0.003} 
submission(X=X , y=y , X_test=X_test , params=param , file_name="DNN_mix",scheduler= True,)


### Nested Cross Validation

In [ ]:
def nested_cross_validation(X,y,params, num_outer_folds=3, num_inner_folds=3, learning_rate=0.001, lr_modifier = False):
    print(f"NESTED CV : {num_outer_folds} outer folds | {num_inner_folds} inner fold \n____________________________________________________________")
    
    X  = torch.tensor(X.to_numpy(), dtype=torch.float32) 
    y = torch.tensor(y.to_numpy(), dtype=torch.float32).view(-1, 1)
    outer_kf = KFold(n_splits=num_outer_folds, shuffle=True, random_state=seed_num)
    
    i = 1
    all_best_param = []
    test_score = []
    train_score = []
    all_test_indices = []
    all_train_indices = []
    #for train_index, test_index in outer_kf.split(X):
    for train_index, test_index in tqdm(outer_kf.split(X), desc="Outer K-Fold", total=outer_kf.get_n_splits(X), colour = "green"):
        all_test_indices.extend(test_index)
        all_train_indices.extend(train_index)
        print(f"OUTER FOLD:{i})")
            
        X_train_outer, X_test_outer = X[train_index], X[test_index]
        y_train_outer, y_test_outer = y[train_index], y[test_index]
        outer_test = []
        outer_train = []
        inner_kf = KFold(n_splits=num_inner_folds, shuffle=True, random_state=seed_num)
        t = 0 
        for param_combination in params:
            
            inner_test = []
            inner_train = []
            for inner_train_index, inner_val_index in inner_kf.split(X_train_outer):
                X_train_inner, X_val_inner = X_train_outer[inner_train_index], X_train_outer[inner_val_index]
                y_train_inner, y_val_inner = y_train_outer[inner_train_index], y_train_outer[inner_val_index]

                
                accuracy = simple_validation(X=X_train_inner,y= y_train_inner,X_val= X_val_inner, y_val=y_val_inner,
                                                params=param_combination, learning_rate=learning_rate, nested_cross_validation= True,verbose=True, lr_modifier= lr_modifier)
                inner_train.append(accuracy[0])
                inner_test.append(accuracy[1])

            mean_inner_test = np.mean(inner_test)
            mean_inner_train = np.mean(inner_train)
            
            outer_test.append(mean_inner_test)
            outer_train.append(mean_inner_train)
            
            print(f"    Param {t}) mean RMSE valid: {mean_inner_test} | train:{mean_inner_train}")
            t+=1
        best_param_combination = params[np.argmin(outer_test)]
        all_best_param.append(best_param_combination)
        print(f"    Best param : {all_best_param.index(best_param_combination) +1}")
        
        
        train_test_accuracy = simple_validation(X=X_train_outer,y= y_train_outer,X_val= X_test_outer, y_val=y_test_outer,
                                                params=best_param_combination, learning_rate=learning_rate, nested_cross_validation= True,verbose=True, lr_modifier= lr_modifier)
        train_score.append(train_test_accuracy[0])
        test_score.append(train_test_accuracy[1])
        i += 1
        print(f"RMSE test: {train_test_accuracy[1]} | train: {train_test_accuracy[0]} \n____________________________________________________________")
    
    
    counters = {key: Counter(param[key] for param in all_best_param) for key in all_best_param[0]}
    most_frequent_values = {key: counter.most_common(1)[0][0] for key, counter in counters.items()}
    print(f"|BEST PARAM:{most_frequent_values}\n")
    return most_frequent_values


param = [                                   
    {'n1': 1024, 'n2': 2048, 'activation': 'tanh', "num_epochs" : 300, "batch_size" : 64, "dropout_rate" : 0.3, "lr_max" : 0.0035},
    {'n1': 1024, 'n2': 2048, 'activation': 'tanh', "num_epochs" : 300, "batch_size" : 64, "dropout_rate" : 0.3},
    {'n1': 1024, 'n2': 1024, 'activation': 'tanh', "num_epochs" : 300, "batch_size" : 64, "dropout_rate" : 0.3},
    {'n1': 1024, 'n2': 2048, 'activation': 'tanh', "num_epochs" : 300, "batch_size" : 64, "dropout_rate" :0.3, "opti" : "adamW"},
    {'n1': 1024, 'n2': 2048, 'activation': 'tanh', "num_epochs" : 300, "batch_size" : 64, "dropout_rate" : 0.3, "lr_max" : 0.0025 },
    ]


nested_cross_validation(X,y,param,3,3, lr_modifier=True)



Missing test CDDD datas resolving for Kaggle submission


In [8]:

def missing_cddd(file_to_change, final_file_name = None) : 
    if final_file_name == None: 
        final_file_name == file_to_change
    datas_folder = "prediction/"
    missing_test_cddd = [279, 462, 695, 883, 913, 1202]
    sub_cddd = pd.read_csv(f"prediction/{file_to_change}.csv")
    sub_ecfp= pd.read_csv("prediction\DNN_ECFP.csv")
    sub_cddd.iloc[missing_test_cddd] = sub_ecfp.iloc[missing_test_cddd].values
    sub_cddd.to_csv(f"prediction/{final_file_name}.csv".format(datas_folder), index=False)
    print("_____________________\n|Complete Done :D")
    return 0 


_____________________
|Complete Done :D


0